# Ngrams lab
September 17
Building n-gram model of Phantom of the Opera text (txt) file. 

In [2]:
# PART 1: setting up and importing 
import numpy as np
import re

import nltk
# if you haven't downloaded punkt before, you only need to run the line below once 
nltk.download('punkt')
from nltk import word_tokenize
from nltk import sent_tokenize

from nltk.util import bigrams
from nltk.lm.preprocessing import padded_everygram_pipeline

import requests

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ismer\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [40]:
# you will need to leverage the requests package
r = requests.get(r'https://www.gutenberg.org/files/175/175.txt')
phantom_opera = r.text

# first, remove unwanted new line and tab characters from the text
for char in ["\n", "\r", "\d", "\t"]:
    phantom_opera = phantom_opera.replace(char, " ")

# check
print(phantom_opera[:100])

The Project Gutenberg EBook of The Phantom of the Opera, by Gaston Leroux    This eBook is for the u


In [49]:
# remove the metadata at the beginning - this is slightly different for each book
phantom_opera = phantom_opera [985:]

In [53]:
# PART 2: creating bigram
# 2 is for bigrams
n = 2
#specify the text you want to use
text = phantom_opera

In [54]:
# step 1: tokenize the text into sentences
sentences = nltk.sent_tokenize(text)

# step 2: tokenize each sentence into words
tokenized_sentences = [nltk.word_tokenize(sent) for sent in sentences]

# step 3: convert each word to lowercase
tokenized_text = [[word.lower() for word in sent] for sent in tokenized_sentences]

#notice the sentence breaks and what the first 10 items of the tokenized text
print(tokenized_text[0])

['be', 'difficult', 'to', 'find', 'at', 'the', 'present', 'day', ',', 'in', 'the', 'foyer', 'of', 'the', 'ballet', ',', 'old', 'men', 'of', 'the', 'highest', 'respectability', ',', 'men', 'upon', 'whose', 'word', 'one', 'could', 'absolutely', 'rely', ',', 'who', 'would', 'remember', 'as', 'though', 'they', 'happened', 'yesterday', 'the', 'mysterious', 'and', 'dramatic', 'conditions', 'that', 'attended', 'the', 'kidnapping', 'of', 'christine', 'daae', ',', 'the', 'disappearance', 'of', 'the', 'vicomte', 'de', 'chagny', 'and', 'the', 'death', 'of', 'his', 'elder', 'brother', ',', 'count', 'philippe', ',', 'whose', 'body', 'was', 'found', 'on', 'the', 'bank', 'of', 'the', 'lake', 'that', 'exists', 'in', 'the', 'lower', 'cellars', 'of', 'the', 'opera', 'on', 'the', 'rue-scribe', 'side', '.']


In [57]:
# notice what the first 10 items are of the vocabulary - changed to 13 to print full word
print(text[:13])

 be difficult


In [58]:
# we imported this function from nltk
train_data, padded_sents = padded_everygram_pipeline(n, tokenized_text)

In [59]:
from nltk.lm import MLE
# we imported this function from nltk linear models (lm) 
# it is for Maximum Likelihood Estimation

# MLE is the model we will use
lm = MLE(n)

In [60]:
# currently the vocab length is 0: it has no prior knowledge
len(lm.vocab)

0

In [61]:
# fit the model 
# training data is the bigrams and unigrams 
# the vocab is all the sentence tokens in the corpus 

lm.fit(train_data, padded_sents)
len(lm.vocab)

7110

In [62]:
# inspect the model's vocabulary. 
# be sure that a sentence you know exists (from tokenized_text) is in the 
print(lm.vocab.lookup(tokenized_text[0]))

('be', 'difficult', 'to', 'find', 'at', 'the', 'present', 'day', ',', 'in', 'the', 'foyer', 'of', 'the', 'ballet', ',', 'old', 'men', 'of', 'the', 'highest', 'respectability', ',', 'men', 'upon', 'whose', 'word', 'one', 'could', 'absolutely', 'rely', ',', 'who', 'would', 'remember', 'as', 'though', 'they', 'happened', 'yesterday', 'the', 'mysterious', 'and', 'dramatic', 'conditions', 'that', 'attended', 'the', 'kidnapping', 'of', 'christine', 'daae', ',', 'the', 'disappearance', 'of', 'the', 'vicomte', 'de', 'chagny', 'and', 'the', 'death', 'of', 'his', 'elder', 'brother', ',', 'count', 'philippe', ',', 'whose', 'body', 'was', 'found', 'on', 'the', 'bank', 'of', 'the', 'lake', 'that', 'exists', 'in', 'the', 'lower', 'cellars', 'of', 'the', 'opera', 'on', 'the', 'rue-scribe', 'side', '.')


In [64]:
# see what happens when we include a word that is not in the vocab. 
print(lm.vocab.lookup('be difficult to find at the tiktok .'.split()))

('be', 'difficult', 'to', 'find', 'at', 'the', '<UNK>', '.')


In [69]:
# how many times does phantom appear in the model?
print(lm.counts['phantom'])

# what is the probability of phantom appearing? 
# this is technically the relative frequency of phantom appearing 
lm.score('phantom')

4


3.365785111449559e-05

In [70]:
# how often does (phantom, and) occur and what is the relative frequency?
print(lm.counts[['phantom']]['and'])
lm.score('and', 'phantom'.split())

0


0.0

In [71]:
# what is the score of 'UNK'? 

lm.score("<UNK>")

0.0

In [76]:
# generate a 20 word sentence starting with the word, 'ghost'

print(lm.generate(20, text_seed= 'ghost', random_seed=42))

['my', 'allowance', '.', '</s>', 'render', 'playful', '.', "''", '</s>', "'s", 'death', '.', "''", '</s>', 'no', 'longer', 'against', 'the', 'stalls', ',']


In [77]:
from nltk.tokenize.treebank import TreebankWordDetokenizer

detokenize = TreebankWordDetokenizer().detokenize

def generate_sent(lm, num_words, text_seed, random_seed=42):
    """
    :param model: An ngram language model from `nltk.lm.model`.
    :param num_words: Max no. of words to generate.
    :param random_seed: Seed value for random.
    """
    content = []
    for token in lm.generate(num_words, text_seed=text_seed, random_seed=random_seed):
        if token == '<s>':
            continue
        if token == '</s>':
            break
        content.append(token)
    return detokenize(content)

In [78]:
# Now generate sentences that look much nicer. 
generate_sent(lm, 20, text_seed='ghost', random_seed = 42)

'my allowance.'

In [83]:
# Generating more sentences
generate_sent(lm, 50, text_seed='They', random_seed = 42)

'my allowance.'